# Package, module setting

In [1]:
import torch
from torch.utils.tensorboard import SummaryWriter
import sys
import time
import yaml
import torchvision
from torchvision import transforms, datasets

import numpy as np
import os
from sklearn import preprocessing
from torch import cuda
from torch.utils.data.dataloader import DataLoader
from tqdm import tqdm

In [2]:
sys.path.append('../')
from mymodels.resnet_base_network import ResNet18
from mydata.imageloader import MyDataset, psnrDataUnit

In [3]:
batch_size = 32
data_transforms = torchvision.transforms.Compose([transforms.ToTensor()])

config = yaml.load(open("../config/config.yaml", "r"), Loader=yaml.FullLoader)
writer = SummaryWriter()

# device = 'cpu'
device = 'cuda' if cuda.is_available() else 'cpu'
print(f"Training with: {device}")
if device=='cuda':
    torch.cuda.empty_cache()

Training with: cuda


# Train-data, Test-data
- shuffle the train data
- dataset
- dataloader

# Encoder loading
load encoder for both images

In [4]:
encoder = ResNet18(**config['network'])
output_feature_dim = encoder.projetion.net[0].in_features
print(output_feature_dim)

512


# Load encoded tensors

In [5]:
# load tensor array
train_load_path = "./tensors/run1111_0505/train.pt"
test_load_path = "./tensors/run1111_0505/test.pt"

# train_load_path = "./tensors/run1111_0505/train.pt"  # 100,000 data
# test_load_path = "./tensors/run1111_0505/test.pt"   # 100,000 data

loaded1 = torch.load(train_load_path)
loaded2 = torch.load(test_load_path)

x1_train = loaded1['x1']
x2_train = loaded1['x2']
y_train = loaded1['y']
x1_test = loaded2['x1']
x2_test = loaded2['x2']
y_test = loaded2['y']

print("Training data shape:", x1_train.shape, x2_train.shape, y_train.shape)
print("Testing data shape:", x1_test.shape, x2_test.shape, y_test.shape)

Training data shape: torch.Size([101463, 512]) torch.Size([101463, 512]) torch.Size([101463])
Testing data shape: torch.Size([101463, 512]) torch.Size([101463, 512]) torch.Size([101463])


In [6]:
scaler = preprocessing.StandardScaler()
scaler.fit(x1_train)
x1_train = scaler.transform(x1_train).astype(np.float32)
x2_train = scaler.transform(x2_train).astype(np.float32)
x1_test = scaler.transform(x1_test).astype(np.float32)
x2_test = scaler.transform(x2_test).astype(np.float32)

In [7]:
def create_data_loaders_from_arrays(X1_train, X2_train, y_train, X1_test, X2_test, y_test):
    
    train = torch.utils.data.TensorDataset(X1_train, X2_train, y_train)
    train_loader = torch.utils.data.DataLoader(train, batch_size=512, shuffle=True)

    test = torch.utils.data.TensorDataset(X1_test, X2_test, y_test)
    test_loader = torch.utils.data.DataLoader(test, batch_size=1, shuffle=False)
    return train_loader, test_loader

In [8]:
train_loader, test_loader = create_data_loaders_from_arrays(torch.from_numpy(x1_train), torch.from_numpy(x2_train), y_train, \
                                                            torch.from_numpy(x1_test), torch.from_numpy(x2_test), y_test)

# MLP setting
number of hidden layers: 5

In [9]:
# Neural Network Class
class MyOne(torch.nn.Module):
    def __init__(self, D_in, H1, H2, H3, H4, H5, H6, H7, D_out):
        super().__init__()
        self.linear1 = torch.nn.Linear(D_in, H1)
        self.linear2 = torch.nn.Linear(H1, H2)
        self.linear3 = torch.nn.Linear(H2, H3)
        self.linear4 = torch.nn.Linear(H3, H4)
        self.linear5 = torch.nn.Linear(H4, H5)
        self.linear6 = torch.nn.Linear(H5, H6)
        self.linear7 = torch.nn.Linear(H6, H7)
        self.linear8 = torch.nn.Linear(H7, D_out)
    
    def forward(self, x):
        x = torch.nn.functional.relu(self.linear1(x))
        x = torch.nn.functional.relu(self.linear2(x))
        x = torch.nn.functional.relu(self.linear3(x))
        x = torch.nn.functional.relu(self.linear4(x))
        x = torch.nn.functional.relu(self.linear5(x))
        x = torch.nn.functional.relu(self.linear6(x))
        x = torch.nn.functional.relu(self.linear7(x))
        x = self.linear8(x)
        return x    

In [10]:
class LastOne(torch.nn.Module):
    def __init__(self, D_in, H1, D_out):
        super().__init__()
        self.linear1 = torch.nn.Linear(D_in, H1)
        self.linear2 = torch.nn.Linear(H1, D_out)
    
    def forward(self, x):
        x = torch.nn.functional.relu(self.linear1(x))
        x = self.linear2(x)
        return x

In [11]:
# logreg = LogisticRegression(output_feature_dim*2, 10)
# logreg = logreg.to(device)
# mymo = MyOne(output_feature_dim, output_feature_dim, output_feature_dim, output_feature_dim, output_feature_dim, output_feature_dim, 256)

# hidden 7
mymo = MyOne(output_feature_dim, 512, 512, 256, 256, 128, 128, 64, 64)
mymo = mymo.to(device)

# # hidden 3
# mymo = MyOne(output_feature_dim, 256, 128, 64, 16)
# mymo = mymo.to(device)

# 모델의 state_dict 출력
print("Model's state_dict:")
for param_tensor in mymo.state_dict():
    print(param_tensor, "\t", mymo.state_dict()[param_tensor].size())

Model's state_dict:
linear1.weight 	 torch.Size([512, 512])
linear1.bias 	 torch.Size([512])
linear2.weight 	 torch.Size([512, 512])
linear2.bias 	 torch.Size([512])
linear3.weight 	 torch.Size([256, 512])
linear3.bias 	 torch.Size([256])
linear4.weight 	 torch.Size([256, 256])
linear4.bias 	 torch.Size([256])
linear5.weight 	 torch.Size([128, 256])
linear5.bias 	 torch.Size([128])
linear6.weight 	 torch.Size([128, 128])
linear6.bias 	 torch.Size([128])
linear7.weight 	 torch.Size([64, 128])
linear7.bias 	 torch.Size([64])
linear8.weight 	 torch.Size([64, 64])
linear8.bias 	 torch.Size([64])


# Model training1

In [12]:
optimizer = torch.optim.Adam(mymo.parameters(), lr=3e-4)
# criterion = torch.nn.CrossEntropyLoss()
criterion = torch.nn.L1Loss()
# criterion = torch.nn.MSELoss()
eval_every_n_epochs = 20
first_epoch = 400
mymo.train()
temp1, temp2, temp3 = 1, 2, 3
check = 1;
# device = 'cuda' if cuda.is_available() else 'cpu'
# print(f"Training with: {device}")
for epoch in tqdm(range(first_epoch)):
#     train_acc = []
    for x1, x2, y in train_loader:

        x1 = x1.to(device)
        x2 = x2.to(device)
        y = y.to(device)
        
        temp1 = x1
        temp2 = x2
        temp3 = y
        # zero the parameter gradients
        optimizer.zero_grad() 
        
        out1 = mymo(x1)
        out2 = mymo(x2)
        # predictions = torch.argmax(logits, dim=1)
        
        # out = torch.cat([out1, out2], dim=1)
        # logits = lastlayer(out)
        logits = torch.cdist(out1.unsqueeze(1), out2.unsqueeze(1))
        
        # if epoch % eval_every_n_epochs == 0 and check == 1:
        #     print(temp1.shape, temp2.shape, temp3.shape, out1.shape, out.shape)
        #     print(logits.squeeze(1).shape)
        #     print(y.unsqueeze(1).shape)
        #     check = 0


        loss = criterion(logits.squeeze(1), y.unsqueeze(1))
        # loss = criterion(logits, y.unsqueeze(1))
        # loss = criterion(logits.squeeze(1), y.unsqueeze(1))

        loss.backward()
        optimizer.step()
        writer.add_scalar("Loss/train", loss, epoch)
    
writer.close()

  0%|          | 0/400 [00:01<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
test_result_diff = []

now = time.localtime()

with torch.no_grad():
    mymo.eval()
    for x1, x2, y in tqdm(test_loader):
        x1 = x1.to(device)
        x2 = x2.to(device)
        y = y.to(device)    
        
        out1 = mymo(x1)
        out2 = mymo(x2)
        # predictions = torch.argmax(logits, dim=1)
        
        logits = torch.cdist(out1.unsqueeze(1), out2.unsqueeze(1))
        logits = logits.squeeze(1)

        ty = 1/float(y[0].item())
        
        if logits[0].item()==0:
            oy = 100
        else: 
            oy = 1 /logits[0].item()
        test_result_diff.append((ty, oy, abs(ty-oy)))

with open("result_%.2f_%02d%02d_%02d%02d.txt" %(1.0, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min), 'w') as f:
    for item1, item2, item3 in test_result_diff:
        f.write("sr_sum:%s, model_out:%s, diff(abs):%s\n" % (item1, item2, item3))    